# Proyecto X

## Lectura de los datos y exploración de los encabezados de los archivos.

In [1]:
import pandas as pd

In [2]:
geolocation = pd.read_csv('/content/Copia de ecommerce_geolocation_dataset.csv')
customers = pd.read_csv('/content/Copia de ecommerce_customers_dataset.csv')
order_items = pd.read_csv('/content/Copia de ecommerce_order_items_dataset.csv')
order_payments = pd.read_csv('/content/Copia de ecommerce_order_payments_dataset.csv')
order_reviews = pd.read_csv('/content/Copia de ecommerce_order_reviews_dataset.csv')
orders = pd.read_csv('/content/Copia de ecommerce_orders_dataset.csv')
products = pd.read_csv('/content/Copia de ecommerce_products_dataset.csv')
sellers = pd.read_csv('/content/Copia de ecommerce_sellers_dataset.csv')
category_names = pd.read_csv('/content/Copia de product_category_name_translation.csv')

In [ ]:
print(geolocation.columns.tolist())

['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']


In [ ]:
print(customers.columns.tolist())

['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']


In [ ]:
print(order_items.columns.tolist())

['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']


In [ ]:
print(order_payments.columns.tolist())

['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']


In [ ]:
print(order_reviews.columns.tolist())

['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']


In [ ]:
print(orders.columns.tolist())

['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']


In [ ]:
print(products.columns.tolist())

['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']


In [ ]:
print(sellers.columns.tolist())

['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']


In [ ]:
print(category_names.columns.tolist())

['product_category_name', 'product_category_name_english']


## BBT1-14 Creación de la base de datos y carga de datos

In [3]:
import sqlite3 as sql


#Conexion a la base de datos
conn = sql.connect('ecommerce.db')
cursor = conn.cursor()

#script de creacion de tablas
script = '''
CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_name_lenght REAL,
    product_description_lenght REAL,
    product_photos_qty REAL,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL
);
CREATE TABLE order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id)
);
CREATE TABLE order_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    PRIMARY KEY(order_id,payment_sequential)
);
CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp TEXT,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT
);
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix INTEGER,
    customer_city TEXT,
    customer_state TEXT
);
CREATE TABLE geolocation(
  geolocation_zip_code_prefix INTEGER PRIMARY KEY,
  geolocation_lat REAL,
  geolocation_lng REAL,
  geolocation_city TEXT,
  geolocation_state TEXT
);
CREATE TABLE order_review(
  review_id TEXT,
  order_id TEXT,
  review_score INT,
  review_comment_title TEXT,
  review_comment_message TEXT,
  review_creation_date TEXT,
  review_answer_timestamp TEXT
  PRIMARY KEY(order_id,review_id)
);
CREATE TABLE sellers(
  seller_id TEXT PRIMARY KEY,
  seller_zip_code_prefix INTEGER,
  seller_city TEXT,
  seller_state TEXT
);
CREATE TABLE category(
  product_category_name TEXT,
  product_category_name_english TEXT
);
'''
# cursor.executescript(script)
conn.commit()


Inserción de los datos de los csv a la base de datos

In [4]:

geolocation.to_sql('geolocation',conn, index=True, if_exists='replace')
customers.to_sql('customers',conn, index=True, if_exists='replace')
order_items.to_sql('order_items',conn, index=True, if_exists = 'replace')
order_payments.to_sql('order_payments',conn, index=True, if_exists = 'replace')
order_reviews.to_sql('order_review',conn, index=True, if_exists = 'replace')
orders.to_sql('orders',conn, index=True, if_exists = 'replace')
products.to_sql('products',conn, index=True, if_exists = 'replace')
sellers.to_sql('sellers',conn, index=True, if_exists = 'replace')
category_names.to_sql('category',conn, index=True, if_exists = 'replace')

71

## BBT1-18 - Extracción de datos desde las tablas customers y orders

In [ ]:
query_1 = '''
SELECT c.*, o.*
FROM customers AS c
JOIN orders as o ON c.customer_id = o.customer_id;

'''
df_customers_orders = pd.read_sql_query(query_1, conn)



## BBT1-19  Extracción de datos desde las tablas orders y orders_items

In [ ]:
query_2 = '''
SELECT o.*, oi.*
FROM orders AS o
JOIN order_items AS oi ON o.order_id = oi.order_id;
'''
df_orders_ordersitems = pd.read_sql_query(query_2,conn)

## BBT1-20  Extracción de datos desde las tablas orders_items y sellers

In [ ]:
query_3 = '''
SELECT oi.*, s.*
FROM order_items AS oi
JOIN sellers AS s ON oi.seller_id = s.seller_id;
'''
df_orderitems_sellers = pd.read_sql_query(query_3, conn)

## BBT1-23 Extracción de datos desde las tablas customers y geolocations

In [15]:
# Crear un DataFrame que contenga el JOIN de la tabla customers y tabla geolocation para extraer los datos de customers con sus latitudes y longitudes

query_bbt1_23 = '''
  SELECT customer_unique_id, customer_zip_code_prefix as Zip_code, geolocation_lat as Latitud, geolocation_lng as Longitud
  FROM customers c
  JOIN geolocation g
  ON g.geolocation_zip_code_prefix = c.customer_zip_code_prefix
  LIMIT 10
  '''
cust_geo = pd.read_sql_query(query_bbt1_23, conn)
cust_geo

,customer_unique_id,Zip_code,Latitud,Longitud
0,861eff4711a542e4b93843c6dd7febb0,14409,-20.519565,-47.391123
1,861eff4711a542e4b93843c6dd7febb0,14409,-20.518938,-47.397374
2,861eff4711a542e4b93843c6dd7febb0,14409,-20.518888,-47.394047
3,861eff4711a542e4b93843c6dd7febb0,14409,-20.518675,-47.393997
4,861eff4711a542e4b93843c6dd7febb0,14409,-20.518012,-47.397669
5,861eff4711a542e4b93843c6dd7febb0,14409,-20.517861,-47.396860
6,861eff4711a542e4b93843c6dd7febb0,14409,-20.517802,-47.396757
7,861eff4711a542e4b93843c6dd7febb0,14409,-20.517560,-47.383913
8,861eff4711a542e4b93843c6dd7febb0,14409,-20.517125,-47.394045
9,861eff4711a542e4b93843c6dd7febb0,14409,-20.516322,-47.396535


## BBT1-24 Extracción de datos desde las tablas orders y geolocations

In [13]:
# Crear un DataFrame que contenga el JOIN de la tabla orders y tabla geolocation para extraer los datos de órdenes con sus latitudes y longitudes

query_bbt1_24 = '''
  SELECT order_id, customer_zip_code_prefix as Zip_code, geolocation_lat as Latitud, geolocation_lng as Longitud
  FROM customers c
  JOIN orders o
  ON c.customer_id = o.customer_id
  JOIN geolocation g
  ON g.geolocation_zip_code_prefix = c.customer_zip_code_prefix
  LIMIT 10
  '''
orders_geo = pd.read_sql_query(query_bbt1_24, conn)
orders_geo

,order_id,Zip_code,Latitud,Longitud
0,00e7ee1b050b8499577073aeb2a297a1,14409,-20.519565,-47.391123
1,00e7ee1b050b8499577073aeb2a297a1,14409,-20.518938,-47.397374
2,00e7ee1b050b8499577073aeb2a297a1,14409,-20.518888,-47.394047
3,00e7ee1b050b8499577073aeb2a297a1,14409,-20.518675,-47.393997
4,00e7ee1b050b8499577073aeb2a297a1,14409,-20.518012,-47.397669
5,00e7ee1b050b8499577073aeb2a297a1,14409,-20.517861,-47.396860
6,00e7ee1b050b8499577073aeb2a297a1,14409,-20.517802,-47.396757
7,00e7ee1b050b8499577073aeb2a297a1,14409,-20.517560,-47.383913
8,00e7ee1b050b8499577073aeb2a297a1,14409,-20.517125,-47.394045
9,00e7ee1b050b8499577073aeb2a297a1,14409,-20.516322,-47.396535


## BBT1-25 Extracción de datos desde las tablas orders y reviews

In [11]:
#Crear un DataFrame que contenga el JOIN de la tabla orders y tabla reviews para extraer los datos de órdenes con sus scores

query_bbt1_25 = '''
  SELECT o.order_id, review_score
  FROM orders o
  JOIN order_review r
  ON o.order_id = r.order_id
  ORDER BY review_score
  LIMIT 10
  '''
orders_score = pd.read_sql_query(query_bbt1_25, conn)
orders_score

,order_id,review_score
0,76c6e866289321a7c93b82b54852dc33,1
1,e6ce16cb79ec1d90b1da9085a6118aeb,1
2,acce194856392f074dbf9dada14d8d82,1
3,1790eea0b567cf50911c057cf20f90f9,1
4,6ea2f835b4556291ffdc53fa0b3b95e8,1
5,ee64d42b8cf066f35eac1cf57de1aa85,1
6,6ebaec694d7025e2ad4a05dba887c032,1
7,9faeb9b2746b9d7526aef5acb08e2aa0,1
8,66e4624ae69e7dc89bd50222b59f581f,1
9,a685d016c8a26f71a0bb67821070e398,1
